## Import

In [100]:
from ultralytics import YOLO, settings
settings.update({"runs_dir": "../runs", "weights_dir": "../weights", "datasets_dir": "../data/datasets"})
YOLO_try = False
import cv2
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn import neighbors
import torch
from torch import nn
import torch.nn.functional as functional
import copy
from matplotlib import cm
from matplotlib.colors import Normalize
import open3d as o3d
from torch.utils.data import Dataset, DataLoader
import json
import time
from matplotlib.lines import Line2D
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.datasets import ImageFolder
from torchvision import transforms
import torchvision.transforms as T
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import matplotlib.patches as patches

## Creating Dataset

In [147]:
class CellDatabase(Dataset):
    def __init__(self, folder_path, jpg_shape=(1530, 1369), gauss_p=0.8, s_and_p_p=0.8, speckle_p=0.8):
        self.xs = []
        self.ys = []
        if not os.path.exists(folder_path):
            raise Exception("Folder not found, dataset not created!")
        subdirs = os.listdir(folder_path)
        if not ('annotations' in subdirs and 'images' in subdirs):
            raise Exception("Folder not in right standard (YOLO), dataset not created!")
        annotations = (sorted(os.listdir(os.path.join(folder_path, 'annotations'))))
        images = (sorted( os.listdir(os.path.join(folder_path, 'images'))))
        for img_path, ann_path in zip(images, annotations):
            x = self.padding_img(torchvision.io.read_image(os.path.join(folder_path, 'images', img_path)), jpg_shape, 0)

            with open(os.path.join(folder_path, 'annotations', ann_path), "r") as file: 
                labels, boxes = [], []
                for line in file:
                    content = line.split(' ')
                    labels.append(int(content[0]))
                    centre_x, centre_y, width, height = [float(element) for element in content[1:]]
                    x_min = centre_x - (width / 2)
                    y_min = centre_y - (height / 2)
                    x_max = centre_x + (width / 2)
                    y_max = centre_y + (height / 2)
                    boxes.append([x_min, y_min, x_max, y_max])
            y = {'labels': torch.tensor(labels, dtype=torch.int64), 'boxes': torch.tensor(boxes, dtype=torch.float32)}
            self.xs.append(x)
            self.ys.append(y)

        print(torch.stack(self.xs).shape)
        mean = tot_sum/tot_pxls
        if np.random.rand() < gauss_p:
            noisy = torch.tensor(self.add_gaussian_noise(x[0], mean(), all_imgs.std()), dtype=torch.float32).unsqueeze(0)
            noisy = (noisy - min_val) / (max_val - min_val + 1e-10)
            self.xs.append(noisy)
            self.ys.append(y)
        if np.random.rand() < s_and_p_p:
            noisy = torch.tensor(self.add_salt_pepper_noise(x[0]), dtype=torch.float32).unsqueeze(0)
            noisy = (noisy - min_val) / (max_val - min_val + 1e-10)
            self.xs.append(noisy)
            self.ys.append(y)
        if np.random.rand() < speckle_p:
            noisy = torch.tensor(self.add_speckle_noise(x[0]), dtype=torch.float32).unsqueeze(0)
            noisy = (noisy - min_val) / (max_val - min_val + 1e-10)
            self.xs.append(noisy)
            self.ys.append(y)
    
    def __len__(self):
        return len(self.xs)
    
    def __getitem__(self, idx):
        y = self.ys[idx]
        x = self.xs[idx]
        return x, y

    def show_x(self, idx, annotate_ys=True):
        if annotate_ys:
            self.mark_bb_on_img(self.xs[idx], self.ys[idx])
        else:
            plt.imshow(self.xs[idx].numpy().transpose(1, 2, 0))
        
    def print_y(self, idx):
        print(df.ys[idx])
    
    @staticmethod
    def padding_img(img, desired_shape, padding_value):
        padded_channels = []
    
        # Calculate padding for height and width
        pad_height_top = (desired_shape[0] - img.shape[1]) // 2  # Padding at the top
        pad_height_bottom = desired_shape[0] - img.shape[1] - pad_height_top  # Padding at the bottom
        pad_width_left = (desired_shape[1] - img.shape[2]) // 2  # Padding on the left
        pad_width_right = desired_shape[1] - img.shape[2] - pad_width_left  # Padding on the right
        
        for channel in img:
            # Padding along height (top and bottom)
            padded_channel = torch.nn.functional.pad(channel, (pad_width_left, pad_width_right, pad_height_top, pad_height_bottom), value=padding_value)
            
            # Add the padded channel to the list
            padded_channels.append(padded_channel)
        
        # Stack all padded channels into one image tensor
        padded_img = torch.stack(padded_channels)
        
        return padded_img


    @staticmethod
    def mark_bb_on_img(x, y):
        plt.figure(figsize=(10,10))
        colors = ['red']
        legend_handle = []
        for i, clr  in enumerate(colors):
            legend_handle.append(Line2D([0], [0], color=clr, marker='s', label=str(i), markersize=5, linestyle='none'))
        ax = plt.subplot(1, 2, 1)
        for j, (label, box) in enumerate(zip(y['labels'], y['boxes'])):
            x_min, y_min, x_max, y_max = box
            ax.add_patch(patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=2, edgecolor=colors[label], facecolor='none'))
        plt.legend(handles=legend_handle, bbox_to_anchor=(1, 1))
        plt.imshow(x.numpy().transpose(1, 2, 0))
          
    @staticmethod
    def normalize_img(img, min_value, max_value):
        return (img - min_value) / (max_value - min_value + 1e-10)

    @staticmethod
    def add_gaussian_noise(image, mean, std):
        return image + np.random.normal(mean, std, image.shape)
    
    @staticmethod
    def add_salt_pepper_noise(image, prob=0.01):
        noisy = copy.deepcopy(image)
        noisy = np.array(noisy)
        noise_prob = np.random.rand(noisy.shape[0], noisy.shape[1])
        noisy[noise_prob < prob / 2] = noisy.min()  # Pepper (black)
        noisy[noise_prob > 1 - prob / 2] = noisy.max()  # Salt (white)
        return noisy

    @staticmethod
    def add_speckle_noise(image, std=0.2):
        img = copy.deepcopy(image)
        img = np.array(img)
        noise = np.random.normal(0, std, img.shape)
        return img + img * noise
    
df = CellDatabase("../data/cells_dataset")

torch.Size([4, 3, 1530, 1369])


NameError: name 'tot_sum' is not defined

## YOLO Try (on hold)

In [2]:
if YOLO_try:
    # Create a new YOLO model from scratch
    # model = YOLO("yolo11n.yaml")

    # Load a pretrained YOLO model (recommended for training)
    model = YOLO("yolo11n.pt")


    # Train the model using the 'coco8.yaml' dataset for 3 epochs
    results = model.train(data="coco8.yaml", epochs=3)

    img = cv2.imread("../data/NMC21700-from-top.jpg")
    print(img.shape)
    results = model.predict(source=img, save=True)
    print(results)


## Faster R-CNN

In [123]:
# Load the pre-trained Faster R-CNN model with a ResNet-50 backbone
model = fasterrcnn_resnet50_fpn(weights=None)

# Number of classes (your dataset classes + 1 for background)
num_classes = 2  # For example, 2 classes + background

# Get the number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# Replace the head of the model with a new one (for the number of classes in your dataset)
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)